In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.data_sources.clob import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

In [ ]:
# Constants
CONNECTOR_NAMES = ["binance_perpetual"]
INTERVAL = "15m"
DAYS = 30
FETCH_CANDLES = True
# Features configuration
QUOTE_ASSET = "USDT"
MIN_NOTIONAL_SIZE = 10
VOLATILITY_WINDOW = 100
VOLUME_SHORT_WINDOW = 20
VOLUME_LONG_WINDOW = 100

# Download data
- Get trading rules
- Get candles for the last x days

In [ ]:
import asyncio

trading_pairs_by_exchange = {}
for connector_name in CONNECTOR_NAMES:
    trading_rules = await clob.get_trading_rules(connector_name)
    trading_pairs = trading_rules.filter_by_quote_asset(QUOTE_ASSET)\
        .filter_by_min_notional_size(Decimal(MIN_NOTIONAL_SIZE))\
        .get_all_trading_pairs()
    trading_pairs_by_exchange[connector_name] = set(trading_pairs)

In [ ]:
trading_pairs = list(set.intersection(*trading_pairs_by_exchange.values()))

In [ ]:
trading_pairs

In [ ]:
BATCH_CANDLES_REQUEST = 100
SLEEP_REQUEST = 2.0


if FETCH_CANDLES:
    number_of_calls = (len(trading_pairs) // BATCH_CANDLES_REQUEST) + 1

    all_candles = {}

    for i in range(number_of_calls):
        print(f"Batch {i + 1}/{number_of_calls}")
        start = i * BATCH_CANDLES_REQUEST
        end = (i + 1) * BATCH_CANDLES_REQUEST
        print(f"Start: {start}, End: {end}")
        end = min(end, len(trading_pairs))
        trading_pairs_batch = trading_pairs[start:end]

        tasks = [clob.get_candles_last_days(
        connector_name=CONNECTOR_NAMES[0],
        trading_pair=trading_pair,
        interval=INTERVAL,
        days=DAYS,
        ) for trading_pair in trading_pairs_batch]

        candles = await asyncio.gather(*tasks)
        candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}
        all_candles.update(candles)
        if i != number_of_calls - 1:
            print(f"Sleeping for {SLEEP_REQUEST} seconds")
            await asyncio.sleep(SLEEP_REQUEST)
    clob.dump_candles_cache(os.path.join(root_path, "data"))
else:
    clob.load_candles_cache(os.path.join(root_path, "data"))

In [ ]:
candles = [value for key, value in clob.candles_cache.items() if key[2] == INTERVAL and key[0] == CONNECTOR_NAMES[0] and key[1] in trading_pairs]

In [ ]:
from core.features.candles.volatility import VolatilityConfig
from core.features.candles.volume import VolumeConfig
from research_notebooks.grid_strike.utils import generate_report

report = generate_report(
    candles=candles,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_SHORT_WINDOW, long_window=VOLUME_LONG_WINDOW))
report

In [ ]:
from research_notebooks.grid_strike.utils import filter_top_markets

TOP_X_MARKETS = 5  # Number of top markets to select
VOLUME_THRESHOLD = 0.4
VOLATILITY_THRESHOLD = 0.4

top_markets = filter_top_markets(report_df=report, volume_threshold=0.4, volatility_threshold=0.4)
top_markets_candles = {candle.trading_pair: candle for candle in candles if candle.trading_pair in top_markets["trading_pair"].values}
top_markets

In [ ]:
for i, row in top_markets.head(TOP_X_MARKETS).iterrows():
    fig = top_markets_candles[row["trading_pair"]].fig()
    fig.add_hline(row["sniper_upper_price"], line_dash="dot", line_color="red")
    fig.add_hline(row["sniper_lower_price"], line_dash="dot", line_color="green")
    fig.add_hline(row["grid_upper_price"], line_dash="dot", line_color="blue")
    fig.add_hline(row["grid_lower_price"], line_dash="dot", line_color="blue")
    fig.add_hline(row["grid_mid_price"], line_dash="dot", line_color="blue")
    fig.show()


In [ ]:

from research_notebooks.grid_strike.utils import generate_configs

VERSION = "0.1"
TOTAL_AMOUNT_QUOTE = 1000  # General total amount for all markets
GRID_ALLOCATION = 0.85  # Grid allocation
INVENTORY_BUFFER = 0.02
N_LEVELS = int(TOTAL_AMOUNT_QUOTE * GRID_ALLOCATION / (MIN_NOTIONAL_SIZE * 1.5))  # Number of levels
ACTIVATION_BOUNDS = 0.01  # Input activation bounds


strategy_configs = generate_configs(
    version=VERSION,
    connector_name=CONNECTOR_NAMES[0],
    top_markets=top_markets.head(TOP_X_MARKETS),
    total_amount_quote=TOTAL_AMOUNT_QUOTE,
    activation_bounds=ACTIVATION_BOUNDS,
    grid_allocation=GRID_ALLOCATION,
    inventory_buffer=INVENTORY_BUFFER,
    n_levels=N_LEVELS,
)
strategy_configs

In [ ]:
from core.utils import dump_dict_to_yaml
for config in strategy_configs:
    dump_dict_to_yaml(config=config, folder="configs")

In [ ]:
from dotenv import load_dotenv
from core.services.backend_api_client import BackendAPIClient

load_dotenv()
server = os.getenv("SERVERS").split(",")[-1]
client = BackendAPIClient(host=server)

In [ ]:
client.get_accounts()

In [ ]:
for config in strategy_configs:
    client.add_controller_config(config)

In [ ]:
controller_configs = [f"{config['id']}.yml" for config in strategy_configs]
client.deploy_script_with_controllers(
    bot_name="grid_strike_binance_nikita",
    controller_configs=controller_configs,
    image_name="hummingbot/hummingbot:latest",
    credentials="master_account",
)